In [7]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [8]:
dates_dimension = pd.DataFrame(pd.read_csv("date_dimension.csv"))
weather_dimension = pd.DataFrame(pd.read_csv("weather_dimension.csv"))
mobility_dimension = pd.DataFrame(pd.read_csv("mobility_dimension.csv"))
special_measures_dimension = pd.DataFrame(pd.read_csv("special_measures_dimension.csv"))
phu_dimension = pd.DataFrame(pd.read_csv("phu_dimension.csv"))

cases = pd.read_csv("patients.csv")   
cases_df = pd.DataFrame(cases)    

cases_df

,patient_key,gender,age_group,acquisition_group,outbreak_related,outcome,reporting_phu,reporting_phu_city,onset_date,reported_date,test_date,specimen_date
0,10,MALE,20s,CC,n,Resolved,Peel Public Health,Mississauga,2020-09-07,2020-09-08,2020-09-08,2020-09-07
1,11,FEMALE,50s,NO KNOWN EPI LINK,n,Resolved,Peel Public Health,Mississauga,2020-09-06,2020-09-08,2020-09-08,2020-09-06
2,12,MALE,30s,NO KNOWN EPI LINK,n,Resolved,Peel Public Health,Mississauga,2020-09-04,2020-09-08,2020-09-08,2020-09-07
3,13,MALE,70s,CC,n,Resolved,Peel Public Health,Mississauga,2020-09-06,2020-09-08,2020-09-08,2020-09-06
4,14,FEMALE,70s,CC,n,Resolved,Peel Public Health,Mississauga,2020-09-07,2020-09-08,2020-09-08,2020-09-07
...,...,...,...,...,...,...,...,...,...,...,...,...
107039,108105,FEMALE,70s,NO KNOWN EPI LINK,n,Resolved,Peel Public Health,Mississauga,2020-11-17,2020-11-20,2020-11-20,2020-11-18
107040,108106,FEMALE,20s,NO KNOWN EPI LINK,n,Resolved,Peel Public Health,Mississauga,2020-10-19,2020-10-23,2020-10-23,2020-10-22
107041,108107,MALE,<20,CC,n,Resolved,Halton Region Health Department,Oakville,2020-11-13,2020-11-22,2020-11-22,2020-11-20
107042,108108,FEMALE,80s,OB,y,Fatal,York Region Public Health Services,Newmarket,2020-10-20,2020-10-23,2020-10-23,2020-10-20


In [9]:
#Fact table rows
fact_table_final =  pd.DataFrame(columns=['onset_date_surrogate','reported_date_surrogate','test_date_surrogate','specimen_date_surrogate', 'weather_surrogate','patient_surrogate', 'phu_location_surrogate', 'mobility_surrogate','special_measures_surrogate',"is_fatal", "is_unresolved", "is_resolved"])

fact_table_final

,onset_date_surrogate,reported_date_surrogate,test_date_surrogate,specimen_date_surrogate,weather_surrogate,patient_surrogate,phu_location_surrogate,mobility_surrogate,special_measures_surrogate,is_fatal,is_unresolved,is_resolved


In [10]:
#drop duplicates
patients_data = cases_df.drop_duplicates(subset =['gender', 'age_group', 'acquisition_group', 'outbreak_related','outcome', 'reporting_phu','onset_date','reported_date','test_date','specimen_date']) 
patients_data

,patient_key,gender,age_group,acquisition_group,outbreak_related,outcome,reporting_phu,reporting_phu_city,onset_date,reported_date,test_date,specimen_date
0,10,MALE,20s,CC,n,Resolved,Peel Public Health,Mississauga,2020-09-07,2020-09-08,2020-09-08,2020-09-07
1,11,FEMALE,50s,NO KNOWN EPI LINK,n,Resolved,Peel Public Health,Mississauga,2020-09-06,2020-09-08,2020-09-08,2020-09-06
2,12,MALE,30s,NO KNOWN EPI LINK,n,Resolved,Peel Public Health,Mississauga,2020-09-04,2020-09-08,2020-09-08,2020-09-07
3,13,MALE,70s,CC,n,Resolved,Peel Public Health,Mississauga,2020-09-06,2020-09-08,2020-09-08,2020-09-06
4,14,FEMALE,70s,CC,n,Resolved,Peel Public Health,Mississauga,2020-09-07,2020-09-08,2020-09-08,2020-09-07
...,...,...,...,...,...,...,...,...,...,...,...,...
97912,98880,GENDER DIVERSE,20s,OB,y,Resolved,Toronto Public Health,Toronto,2020-09-29,2020-10-01,2020-10-01,2020-09-29
97950,98919,GENDER DIVERSE,40s,MISSING INFORMATION,n,Resolved,Toronto Public Health,Toronto,2020-11-05,2020-11-06,2020-11-06,2020-11-05
98334,99309,GENDER DIVERSE,<20,MISSING INFORMATION,y,Resolved,Toronto Public Health,Toronto,2020-11-17,2020-11-20,2020-11-20,2020-11-19
101042,102051,FEMALE,90+,OB,n,Resolved,York Region Public Health Services,Newmarket,2020-12-27,2020-12-28,2020-12-28,2020-12-27


In [11]:
#4 months 
start_date = datetime(2020, 9, 1).date()
end_date = datetime(2020, 12, 31).date()

#sort by reported_date
patients_data = patients_data.sort_values('reported_date')
patients_data.head()

regions = ["durham", "halton", "peel", "york", "toronto", "ottawa"]#list of regions

for idx, row in patients_data.iterrows():
    #reported date
    onset_date = row['onset_date']
    reported_date = row['reported_date']
    test_date = row['test_date']
    specimen_date = row['specimen_date']
    phu = row["reporting_phu"]
    
    #link dates key
    #onset_date key
    onset_key = dates_dimension[dates_dimension['date']==onset_date]['test_date_key'].values[0]

    #reported date key
    reported_key = dates_dimension[dates_dimension['date']== reported_date]['test_date_key'].values[0]

    #test date
    test_key = dates_dimension[dates_dimension['date']==test_date]['test_date_key'].values[0]

    #specimen date
    specimen_key = dates_dimension[dates_dimension['date']==specimen_date]['test_date_key'].values[0]

    #link phu
    phu_key = phu_dimension[phu_dimension["phu_name"] == phu]["phu_location_key"].values[0]

    #link regions
    for city in regions:
        if(city in phu.lower()):
            break#return x

    #link weather
    if(city == 'ottawa'):#Ottawa case
        weather_key = weather_dimension[(weather_dimension['station_name'] == 'OTTAWA') & (weather_dimension['date_col'] == test_date)]['weather_surrogate_key'].values[0] 
    else:#GTA case
        weather_key = weather_dimension[(weather_dimension['station_name'] == 'TORONTO') & (weather_dimension['date_col'] == test_date)]['weather_surrogate_key'].values[0] 


    #link mobility
    mobility_key = mobility_dimension[(mobility_dimension['date_col'] == test_date) & ( mobility_dimension['subregion'].str.contains(city, case=False))]['mobility_surrogate_key'].values[0] 


    #link measures
    measures_key = special_measures_dimension[((special_measures_dimension['start_date'] <= test_date) & (special_measures_dimension['end_date'] >= test_date)) & (special_measures_dimension['region'] == city) ]['special_measures_surrogate_key'].values[0]


    #add data to row
    fact_row = []#surrogate key
    fact_row.append(onset_key)#date
    fact_row.append(reported_key)#date 
    fact_row.append(test_key)#date 
    fact_row.append(specimen_key)#date 

    fact_row.append(weather_key)#weather
    fact_row.append(row['patient_key'])#patient_surrogate 
    fact_row.append(phu_key)#(phu_row["phu_location_key"])#phu TODO
    fact_row.append(mobility_key)#mobility
    fact_row.append(measures_key)#measures

    #add measures attributes
    if(row["outcome"].lower() == "fatal"):
        fact_row.append(True)#is_fatal
        fact_row.append(False)#is_unresolved
        fact_row.append(True)#is_resolved
    elif(row["outcome"].lower() == "not resolved"):
        fact_row.append(False)#is_fatal
        fact_row.append(True)#is_unresolved
        fact_row.append(False)#is_resolved
    elif(row["outcome"].lower() == "resolved"):
        fact_row.append(False)#is_fatal
        fact_row.append(False)#is_unresolved
        fact_row.append(True)#is_resolved

    #add row    
    fact_table_final.loc[len(fact_table_final)] = fact_row


Peel Public Health
Peel Public Health
Halton Region Health Department
York Region Public Health Services
York Region Public Health Services
York Region Public Health Services
Peel Public Health
York Region Public Health Services
Peel Public Health
York Region Public Health Services
York Region Public Health Services
York Region Public Health Services
York Region Public Health Services
York Region Public Health Services
York Region Public Health Services
York Region Public Health Services
Peel Public Health
Peel Public Health
Durham Region Health Department
Peel Public Health
York Region Public Health Services
Peel Public Health
Peel Public Health
Peel Public Health
Peel Public Health
Peel Public Health
Peel Public Health
Durham Region Health Department
York Region Public Health Services
Halton Region Health Department
Peel Public Health
Halton Region Health Department
Peel Public Health
Peel Public Health
Peel Public Health
Durham Region Health Department
Durham Region Health Departmen

In [12]:
fact_table_final

,onset_date_surrogate,reported_date_surrogate,test_date_surrogate,specimen_date_surrogate,weather_surrogate,patient_surrogate,phu_location_surrogate,mobility_surrogate,special_measures_surrogate,is_fatal,is_unresolved,is_resolved
0,1,2,2,1,545,618,5,48065,102,False,False,True
1,2,3,3,2,546,798,5,48066,102,False,False,True
2,3,5,5,3,548,598,4,47346,101,False,False,True
3,2,5,5,4,548,678,6,48429,103,False,False,True
4,2,5,5,4,548,683,6,48429,103,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
231,114,119,119,118,662,61244,3,50916,172,False,False,True
232,118,119,119,118,662,102051,6,48543,188,False,False,True
233,117,119,119,117,662,64350,3,50916,172,False,False,True
234,117,121,121,118,664,64450,3,50918,172,False,False,True


In [13]:
fact_table_final.fillna(0, inplace = True)

fact_table_final.to_csv("fact_table_final.csv",index=False)